In [42]:
import pandas as pd
import json
import numpy as np
import re
import datetime
import os
import matplotlib.pyplot as plt

In [43]:
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] #用来正常显示中文标签
desktop = '/Users/panbingqing/Desktop/'

In [44]:
xs_flow = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_history/veh_flow_history.csv')
xs_flow.index = pd.to_datetime(xs_flow.passtime, format="%Y-%m-%dT%H:%M:%S")

In [ ]:
xs_flow.head()

In [46]:
# 每15分钟统计
xs_count = xs_flow.groupby(['camera_id', 'inter_name', 'road_name', 'lane', 'direction']).resample('2T').car_id.count().reset_index()
xs_count.rename(columns={'car_id': 'car_count'}, inplace=True)

In [47]:
xs_count.head()

,camera_id,inter_name,road_name,lane,direction,passtime,car_count
0,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:02:00,4
1,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:04:00,2
2,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:06:00,2
3,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:08:00,6
4,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:10:00,1


In [48]:
xs_count.to_csv(desktop+'xs_count.csv', index=False)

## 清洗异常数据

In [49]:
def get_unusual_point(log_file='/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_inline/log_data.csv'):
    # 日志错误时间点，以2分钟颗粒度进行统计
    log_data = pd.read_csv(log_file)
    log_data.index = pd.to_datetime(
        log_data.log_time, format="%Y-%m-%d %H:%M:%S")
    log_data = log_data[log_data.synopses.isin(['Stream closed', 'Get video failed'])]
    log_count = log_data.groupby('camera_id').resample('15T').log_type.count().reset_index()
    log_count.to_csv(desktop+'log_count.csv', index=False)
    unusual_point = log_count.loc[log_count.log_type > 2,
                                  ['camera_id', 'log_time']]
    return unusual_point.values.tolist()

In [50]:
def clean_unusual(flow_count):
    # 清洗异常点，从2019年5月11日开始
    # 日志错误时间点
    unusual_point = get_unusual_point()
    unusual_point_index = flow_count.apply(
        lambda x: [x['camera_id'], x['passtime']] in unusual_point,
        axis=1)
    flow_count.loc[unusual_point_index, 'car_count'] = np.nan

    # 网络波动异常时间
    unusual_range = [
        {'start_time': datetime.datetime(2019, 5, 17, 10, 30),
         'end_time': datetime.datetime(2019, 5, 17, 17, 0)},
        {'start_time': datetime.datetime(2019, 5, 24, 16, 0),
         'end_time': datetime.datetime(2019, 5, 24, 21, 15)}
    ]

    for _range in unusual_range:
        unusual_range_index = (
            (flow_count.passtime >= _range['start_time']) &
            (flow_count.passtime < _range['end_time'])
        )
        flow_count.loc[unusual_range_index, 'car_count'] = np.nan

    # 特殊错误 1: carmera_id 7 从 2019/5/17 17点球机发生变动，数据异常，截止时间暂缺
    special_unusual = [
        {'camera_id': 7,
         'start_time': datetime.datetime(2019, 5, 17, 17, 0),
         'end_time': datetime.datetime(2019, 5, 30, 17, 30)}
    ]
    for _special in special_unusual:
        special_unusual_index = (
            (flow_count.camera_id == _special['camera_id']) &
            (flow_count.passtime >= _special['start_time']) &
            (flow_count.passtime < _special['end_time'])
        )
        flow_count.loc[special_unusual_index, 'car_count'] = np.nan
    return flow_count

### 异常值填充

In [51]:
def get_fill_data(flow_count):
    # 根据周日期平均值填充缺失值
    flow_count = flow_count.copy()
    flow_count['weekday'] = flow_count.passtime.dt.weekday_name
    flow_count['date'] = flow_count.passtime.dt.date
    flow_count['time'] = flow_count.passtime.dt.time
    
    group_columns = ['camera_id', 'inter_name', 'road_name', 'lane', 'direction', 'weekday']
    date_group = flow_count.groupby(group_columns)
    
    fill_data = pd.DataFrame() 
    for name, group in date_group:
        group = group.pivot_table(index='date', columns='time', values='car_count')
        group.fillna(group.mean(), inplace=True)
        group = group.unstack().reset_index()
        group.rename(columns={0: 'car_count'}, inplace=True)
        
        for i in range(len(group_columns)):
            group[group_columns[i]] = name[i]
            
        fill_data = fill_data.append(group)
        
    return fill_data

In [56]:
clean_data = clean_unusual(xs_count)
#clean_data.to_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_inline/clean_data_history.csv')
clean_data.head()

,camera_id,inter_name,road_name,lane,direction,passtime,car_count
0,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:02:00,4.0
1,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:04:00,2.0
2,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:06:00,2.0
3,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:08:00,6.0
4,1,星沙-滨湖,星沙-滨湖北,1,LB,2019-04-22 00:10:00,1.0


In [57]:
fill_data = get_fill_data(clean_data)
fill_data.to_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_history/fill_data_history.csv')
fill_data.head()

,time,date,car_count,camera_id,inter_name,road_name,lane,direction,weekday
0,00:00:00,2019-04-26,0.0,1,星沙-滨湖,星沙-滨湖北,1,LB,Friday
1,00:00:00,2019-05-03,2.0,1,星沙-滨湖,星沙-滨湖北,1,LB,Friday
2,00:00:00,2019-05-10,0.0,1,星沙-滨湖,星沙-滨湖北,1,LB,Friday
3,00:00:00,2019-05-17,3.0,1,星沙-滨湖,星沙-滨湖北,1,LB,Friday
4,00:00:00,2019-05-24,6.0,1,星沙-滨湖,星沙-滨湖北,1,LB,Friday


In [74]:
fill_data[(fill_data.weekday == 'Friday') & (fill_data.inter_name == '星沙-望仙')]

,time,date,car_count,camera_id,inter_name,road_name,lane,direction,weekday
0,00:00:00,2019-04-19,1.833333,9,星沙-望仙,星沙-望仙东,1,L,Friday
1,00:00:00,2019-04-26,1.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
2,00:00:00,2019-05-03,1.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
3,00:00:00,2019-05-10,1.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
4,00:00:00,2019-05-17,4.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
5,00:00:00,2019-05-24,4.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
6,00:00:00,2019-05-31,0.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
7,00:02:00,2019-04-19,1.166667,9,星沙-望仙,星沙-望仙东,1,L,Friday
8,00:02:00,2019-04-26,0.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
9,00:02:00,2019-05-03,1.000000,9,星沙-望仙,星沙-望仙东,1,L,Friday
